In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [2]:
# load data
 
df=pd.read_csv('Waveform_withoutdupl_norm_v01.csv')  
del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)
df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att13,att14,att15,att16,att17,att18,att19,att20,att21,outlier
0,0.323657,0.793785,0.848661,0.660063,0.622244,0.755102,0.760762,0.671655,0.603723,0.636170,...,0.281899,0.203557,0.286607,0.474383,0.352483,0.358857,0.275626,0.465937,0.220884,1
1,0.423803,0.769774,0.632130,0.572471,0.747495,0.670408,0.715232,0.617958,0.637411,0.457447,...,0.302671,0.375494,0.296429,0.383302,0.545278,0.394286,0.451025,0.470803,0.468541,1
2,0.477504,0.580508,0.554133,0.476538,0.851703,0.509184,0.745861,0.785211,0.508865,0.519149,...,0.341246,0.384387,0.229464,0.350095,0.117819,0.406857,0.149203,0.402676,0.370817,1
3,0.645864,0.570621,0.525029,0.631908,0.682365,0.382653,0.478477,0.608275,0.476950,0.190426,...,0.500495,0.555336,0.463393,0.537951,0.564752,0.485714,0.576310,0.518248,0.597055,1
4,0.219158,0.690678,0.530850,0.715328,0.687375,0.571429,0.649007,0.562500,0.523050,0.303191,...,0.190900,0.408103,0.322321,0.425996,0.400195,0.701714,0.515945,0.294404,0.463186,1


In [4]:
df.shape

(3443, 22)

In [5]:
# data to values
df = df.values

In [6]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/waveform.json")
gc = GCForest(config)

In [7]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,21]
X = df[:,0:21]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
             # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-25 01:04:42,688][cascade_classifier.fit_transform] X_groups_train.shape=[(2306, 21)],y_train.shape=(2306,),X_groups_test.shape=[(1137, 21)],y_test.shape=(1137,)
[ 2018-04-25 01:04:42,689][cascade_classifier.fit_transform] group_dims=[21]
[ 2018-04-25 01:04:42,691][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 01:04:42,693][cascade_classifier.fit_transform] group_ends=[21]
[ 2018-04-25 01:04:42,694][cascade_classifier.fit_transform] X_train.shape=(2306, 21),X_test.shape=(1137, 21)
[ 2018-04-25 01:04:42,696][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2306, 21), X_cur_test.shape=(1137, 21)
[ 2018-04-25 01:04:43,329][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=96.97%
[ 2018-04-25 01:04:44,121][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.40%
[ 2018-04-25 01:04:44,907][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 01:04:50,780][cascade_classifier.transform] X_groups_test.shape=[(1137, 21)]
[ 2018-04-25 01:04:50,782][cascade_classifier.transform] group_dims=[21]
[ 2018-04-25 01:04:50,784][cascade_classifier.transform] X_test.shape=(1137, 21)
[ 2018-04-25 01:04:50,786][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1137, 21)


Test Accuracy of ExtraTrees = 97.361478 %


In [10]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 01:04:52,305][cascade_classifier.transform] X_groups_test.shape=[(1137, 21)]
[ 2018-04-25 01:04:52,307][cascade_classifier.transform] group_dims=[21]
[ 2018-04-25 01:04:52,309][cascade_classifier.transform] X_test.shape=(1137, 21)
[ 2018-04-25 01:04:52,312][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1137, 21)


In [11]:
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 01:04:53,902][cascade_classifier.fit_transform] X_groups_train.shape=[(2306, 23)],y_train.shape=(2306,),X_groups_test.shape=[(1137, 23)],y_test.shape=(1137,)
[ 2018-04-25 01:04:53,905][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-25 01:04:53,907][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 01:04:53,909][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-25 01:04:53,911][cascade_classifier.fit_transform] X_train.shape=(2306, 23),X_test.shape=(1137, 23)
[ 2018-04-25 01:04:53,914][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2306, 23), X_cur_test.shape=(1137, 23)
[ 2018-04-25 01:04:54,562][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.84%
[ 2018-04-25 01:04:55,331][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.84%
[ 2018-04-25 01:04:56,086][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [12]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 01:05:02,004][cascade_classifier.transform] X_groups_test.shape=[(1137, 23)]
[ 2018-04-25 01:05:02,007][cascade_classifier.transform] group_dims=[23]
[ 2018-04-25 01:05:02,008][cascade_classifier.transform] X_test.shape=(1137, 23)
[ 2018-04-25 01:05:02,010][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1137, 23)


Test Accuracy of ExtraTrees = 97.801231 %


In [13]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [14]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-04-25 01:05:03,555][cascade_classifier.fit_transform] X_groups_train.shape=[(2306, 23)],y_train.shape=(2306,),X_groups_test.shape=[(1137, 23)],y_test.shape=(1137,)
[ 2018-04-25 01:05:03,557][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-25 01:05:03,559][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 01:05:03,560][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-25 01:05:03,562][cascade_classifier.fit_transform] X_train.shape=(2306, 23),X_test.shape=(1137, 23)
[ 2018-04-25 01:05:03,564][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2306, 23), X_cur_test.shape=(1137, 23)
[ 2018-04-25 01:05:04,203][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.84%
[ 2018-04-25 01:05:04,964][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.84%
[ 2018-04-25 01:05:05,780][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

('Test Accuracy of ExtraTrees = 97.977133 %', 0, 'fois')


[ 2018-04-25 01:05:13,789][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.84%
[ 2018-04-25 01:05:14,550][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.84%
[ 2018-04-25 01:05:15,376][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.27%
[ 2018-04-25 01:05:16,167][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=96.97%
[ 2018-04-25 01:05:16,931][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.27%
[ 2018-04-25 01:05:17,782][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.84%
[ 2018-04-25 01:05:18,564][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.84%
[ 2018-04-25 01:05:19,370][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.13%


('Test Accuracy of ExtraTrees = 97.977133 %', 1, 'fois')


[ 2018-04-25 01:05:23,416][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.84%
[ 2018-04-25 01:05:24,179][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.84%
[ 2018-04-25 01:05:24,942][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=98.27%
[ 2018-04-25 01:05:25,753][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=96.97%
[ 2018-04-25 01:05:26,530][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.27%
[ 2018-04-25 01:05:27,283][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.84%
[ 2018-04-25 01:05:28,049][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.27%
[ 2018-04-25 01:05:28,805][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.13%


...

('Test Accuracy of ExtraTrees = 98.065084 %', 17, 'fois')


[ 2018-04-25 01:07:52,523][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=96.97%
[ 2018-04-25 01:07:53,332][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.54%
[ 2018-04-25 01:07:54,145][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.40%
[ 2018-04-25 01:07:54,946][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.40%
[ 2018-04-25 01:07:55,815][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.84%
[ 2018-04-25 01:07:56,585][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.40%
[ 2018-04-25 01:07:57,369][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.40%
[ 2018-04-25 01:07:58,185][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.13%


('Test Accuracy of ExtraTrees = 97.977133 %', 18, 'fois')


[ 2018-04-25 01:08:02,111][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=96.97%
[ 2018-04-25 01:08:02,890][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.54%
[ 2018-04-25 01:08:03,656][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.40%
[ 2018-04-25 01:08:04,410][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.84%
[ 2018-04-25 01:08:05,188][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.84%
[ 2018-04-25 01:08:05,963][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.84%
[ 2018-04-25 01:08:06,753][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.84%
[ 2018-04-25 01:08:07,612][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.57%


('Test Accuracy of ExtraTrees = 97.977133 %', 19, 'fois')


# ('Test Accuracy of ExtraTrees = 98.065084 %', 3, 'fois')